## [Running Flask and FastAPI on Google Colab](https://medium.datadriveninvestor.com/flask-on-colab-825d2099d9d8)

In [1]:
!pip install fastapi nest-asyncio pyngrok uvicorn

     |████████████████████████████████| 51kB 4.9MB/s 
     |████████████████████████████████| 747kB 21.8MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 10.1MB 37.6MB/s 
     |████████████████████████████████| 61kB 7.1MB/s 
     |████████████████████████████████| 61kB 7.0MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.0.5-cp37-none-any.whl size=19246 sha256=a2edfb6fe4ca4fb6abfa4d48ba0b083b9162c904b2717e303dca55af5e9d02f9
  Stored in directory: /root/.cache/pip/wheels/0c/13/64/5ebbcc22eaf53fdf5766b397c1fb17c83f5775fdccf0ea1b88
Successfully built pyngrok


In [1]:
from fastapi import FastAPI
import nest_asyncio
from pyngrok import ngrok
import uvicorn

app = FastAPI()

@app.get('/index')
async def home():
  return "Hello World"

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
print('Public docs URL:', ngrok_tunnel.public_url + "/docs")
nest_asyncio.apply()
uvicorn.run(app, port=8000)


INFO:     Started server process [132]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: http://a376a52c482b.ngrok.io
Public docs URL: http://a376a52c482b.ngrok.io/docs
INFO:     124.32.186.1:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     124.32.186.1:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     124.32.186.1:0 - "GET /index HTTP/1.1" 200 OK
INFO:     124.32.186.1:0 - "GET /index HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [132]


## [Deploying Transformer Models](https://chatbotslife.com/deploying-transformer-models-1350876016f)


In [2]:
%%bash
pip install -qq transformers
#pip install torch torchvision
pip install "fugashi[unidic-lite]" 
pip install ipadic

  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-cp37-none-any.whl size=47658825 sha256=661ddf512ca57e62466f5a035a6e5f243b64b357804b387e13469a0a991b0605
  Stored in directory: /root/.cache/pip/wheels/20/48/8d/b66d8361a27f58f41ec86640e4fd2640de0403a6367511eab7
Successfully built unidic-lite
  Created wheel for ipadic: filename=ipadic-1.0.0-cp37-none-any.whl size=13556725 sha256=526baba5fe61ba6ae6ae2a6106c700b3246db67f1e27a53556129970bb8ec0da
  Stored in directory: /root/.cache/pip/wheels/ff/00/d1/0c094a0ce58a77199a0c5801f0ecf510c80f0ecbec27f07d2c
Successfully built ipadic


In [3]:
import torch
from transformers import (
    pipeline,
    AutoModelForSequenceClassification,
    AutoTokenizer
)

class NLP:
    def __init__(self):
        self.gen_tokenizer = AutoTokenizer.from_pretrained('daigo/bert-base-japanese-sentiment', use_fast=False)

    def sentiments(self, text: str):
        if torch.cuda.is_available():
            nlp = pipeline('sentiment-analysis', model="daigo/bert-base-japanese-sentiment", tokenizer=self.gen_tokenizer, device=0)
        else:
            nlp = pipeline('sentiment-analysis', model="daigo/bert-base-japanese-sentiment", tokenizer=self.gen_tokenizer)
        result = nlp(text)[0]
        return f"label: {result['label']}, with score: {round(result['score'], 4)}"


In [4]:
# from nlp import NLP
nlp = NLP()
print(nlp.sentiments("うほほーい、大好き♡"))



label: ポジティブ, with score: 0.9749


In [5]:
print(nlp.sentiments("ケーキ食べ過ぎた、もうだめ。死にたい"))

label: ネガティブ, with score: 0.6268


In [ ]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
#from app.nlp import NLP
import nest_asyncio
from pyngrok import ngrok
import uvicorn

class Message(BaseModel):
    input: str
    output: str = None

app = FastAPI()
nlp = NLP()

origins = [
    "http://localhost",
    "http://localhost:3000",
    "http://127.0.0.1:3000"
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["POST"],
    allow_headers=["*"],
)

@app.post("/sentiment/")
async def sentiment_analysis(message: Message):
    message.output  = str(nlp.sentiments(message.input))
    return {"output" : message.output}

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
print('Public docs URL:', ngrok_tunnel.public_url + "/docs")
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: http://c3ceb9bf82bd.ngrok.io
Public docs URL: http://c3ceb9bf82bd.ngrok.io/docs


```
% curl -X POST "http://9699baf22cae.ngrok.io/sentiment/" -H "accept: application/json" -H "Content-Type: application/json" -d "{\"input\":\"うほほーい、大好き♡\"}"

{"output":"Basil pasta is really nice with the extra salad that I like the best. When I started this recipe this year, I was worried I would make too much pasta with my family but now that I'm on a budget, I'm happy.\n\nNow, the fact of the matter is I'm a fan of pasta with a spicy twist. I'm going to make it spicy this year. I'm going to make it delicious with broccoli, tomato sauce, broccoli crumbles, cheese, cheddar, feta, chives, chili, garlic, cheddar, and onions (a side.)\n\nYou want to make this pasta using your favorite recipe from these two posts, and then add it to a dish with more veggies or other ingredients on top. This is my recipe for a grilled casserole, and I've been using broccoli cauliflower and cauliflower noodles at least 3 times in my pasta routine, so this recipe is a lot easier for me than the simple, easy"}
```